In [0]:
data = [("James", "Sales", 3000),
    ("Michael", "Sales", 4600),
    ("Robert", "Sales", 4100),
    ("Maria", "Finance", 3000),
    ("James", "Sales", 3500),
    ("Scott", "Finance", 3300),
    ("Jen", "Finance", 3900),
    ("Jeff", "Marketing", 3000),
    ("Kumar", "Marketing", 2000),
    ("Saif", "Sales", 4100)
  ]
columns = ["Name","Dept","Salary"]
df = spark.createDataFrame(data=data,schema=columns)
df.select("Name").distinct().show()
print("Distinct Count: " + str(df.select("Name").distinct().count()))

+-------+
|   Name|
+-------+
|  James|
|Michael|
| Robert|
|  Maria|
|  Scott|
|    Jen|
|   Jeff|
|  Kumar|
|   Saif|
+-------+

Distinct Count: 9


In [0]:
import pyspark.sql.functions as fnsql #row_number
from pyspark.sql.window import Window as wnd #window function
from pyspark.sql.types import DateType, StringType, DoubleType, BooleanType

wndspec = wnd.partitionBy("Dept").orderBy("Name")
# df.withColumn("ID", fnsql.concat(string(fnsql.row_number().over(wndspec)),df["Dept"]) ).show()
df.withColumn("ID",\
    # fnsql.concat( fnsql.row_number().cast(StringType()).over(wndspec),df["Dept"] ) ).show()
    fnsql.concat( fnsql.row_number().over(wndspec).cast(StringType() ),fnsql.lit("_"),df["Dept"] )).show()



+-------+---------+------+-----------+
|   Name|     Dept|Salary|         ID|
+-------+---------+------+-----------+
|    Jen|  Finance|  3900|  1_Finance|
|  Maria|  Finance|  3000|  2_Finance|
|  Scott|  Finance|  3300|  3_Finance|
|   Jeff|Marketing|  3000|1_Marketing|
|  Kumar|Marketing|  2000|2_Marketing|
|  James|    Sales|  3000|    1_Sales|
|  James|    Sales|  3500|    2_Sales|
|Michael|    Sales|  4600|    3_Sales|
| Robert|    Sales|  4100|    4_Sales|
|   Saif|    Sales|  4100|    5_Sales|
+-------+---------+------+-----------+



In [0]:
df.select("*")\
    .groupBy("Dept")\
    .agg(fnsql.count("Dept").alias("countx"))\
    .filter(fnsql.col("countx")==3).show()

+-------+------+
|   Dept|countx|
+-------+------+
|Finance|     3|
+-------+------+

